In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.request
import cv2
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

In [3]:
def Get_Stones_Details(driverf):
    time.sleep(1)
    Formula = ""
    Colours = ""
    Elements = ""
    soup = BeautifulSoup(driverf.page_source, 'html.parser')
    try:
        divs = soup.find_all("div",{"class":"collapsesection"})
        for div in divs:
            text = div.text
            EFinder = text.find("Elements ")
            if(EFinder!=-1):
                Elements = (text[EFinder+16:])
                if(Elements.find("-")!=-1):
                    Elements = Elements[:Elements.find("-")]
                else:
                    Elements = Elements[:14]
            CFinder = text.find("Colour:")
            if(CFinder!=-1):
                Colours = text[CFinder+7:]
                if(Colours.find(".")!=-1):
                    Colours = Colours[:Colours.find(".")]
                else:
                    Colours = Colours[:16]
            FFinder = text.find("Formula:")
            if(FFinder!=-1):
                Formula = text[FFinder+8:]
                if(Formula.find("Colour")!=-1):
                    Formula = Formula[:Formula.find("Colour")]
                elif(Formula.find("Elements ")!=-1):
                    Formula = Formula[:Formula.find("Elements ")]
                elif(Formula.find("Formula ")!=-1):
                    Formula = Formula[:Formula.find("Formula ")]
                else:
                    Formula = Formula[:10]

    except NoSuchElementException:
        pass
    except TypeError:
        pass
    except AttributeError:
        pass
    return Formula, Colours, Elements




def Get_Locations(driverf, stone_name):
    time.sleep(2)   
    loc_list = []
    loc_url = []
    final_loc = []
    soup = BeautifulSoup(driverf.page_source, 'html.parser')
    try:
        table = soup.find("table",{"class":"loclisttable"})
        if(len(table)!=0):
            for t in table:
                url_for_cords = t.find_all('a')
                for url in url_for_cords:
                    href = url.get('href')
                    loc_url.append(href)
                divs = t.find_all("div")
                for d in divs:
                    loc_list.append(d.text)


            for loc in loc_url:
                if(loc.find('loc-')!=-1):
                    http = "https://www.mindat.org/"+loc
                    final_loc.append(http)
    except NoSuchElementException:
        pass
    except TypeError:
        pass
    return loc_list,final_loc



def save_photos(df):  
    for ind in df.index:
        i = 0
        image_url_list = df['Image_url'][ind]
        stone_name = df['Name'][ind]
        for image_url in image_url_list:
            FileName = "images.csv"
            FilePath = 'Data\Images\images'
            image_url = 'https://www.mindat.org/'+image_url
            response = requests.get(image_url)
            image_name = FilePath + 'mindat'+stone_name+'ind='+str(ind)+'num='+str(i) + '.jpg' 
            with open(image_name, 'wb') as file:
                file.write(response.content)
            i+=1
            img = cv2.imread(image_name)
    # cv2.imshow(img)



def get_photos(driverf,stone_name):
    time.sleep(1)
    i = 0    
    img_list = []
    try:
        driverf.find_element_by_css_selector("input[type='text']").send_keys(stone_name)
        time.sleep(1)
        driverf.find_element_by_css_selector("input[type='text']").send_keys(Keys.ENTER)
    except ElementClickInterceptedException:
        return img_list
    except NoSuchElementException:
        time.sleep(20)
        driverf.close()
        driverf = webdriver.Chrome(ChromeDriverManager().install())
        driverf.get('https://geology.com/gemstones/')
        pass
    soup = BeautifulSoup(driverf.page_source, 'html.parser')
    try:
        table = soup.find_all("div",{"class":"userbigpicture noborder"})
        for img in table:
            if(i==3):break
            img_list.append(img.find('img')['src'])
            i += 1
    except NoSuchElementException:
        pass
    return img_list

In [27]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driverf = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://geology.com/gemstones/') #getting most popular gemstone's names
driverf.get('https://www.mindat.org/photoscroll.php') # getting thos gemstone's details

soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find("div",{"class":"right"})
Name_list = []

href_list = table.find_all('a')
for href in href_list:
    time.sleep(3)
    data_dict = {}
    Name = href.text
    if(Name.find(" Diamond")!=-1 or Name.find(" Opal")!=-1): continue
    data_dict['Name'] = Name
    image_list = get_photos(driverf, Name) ###### imagesssss
    if(len(image_list)==0): continue
    Formula, Colours, Elements = Get_Stones_Details(driverf)
    data_dict['Formula'] = Formula
    data_dict['Elements'] = Elements
    data_dict['Colours'] = Colours
    data_dict['Image_url'] = image_list
    data_dict['Locations'], data_dict['Locations_url'] = Get_Locations(driverf, Name)
    
    Name_list.append(data_dict)

driver.close()
driverf.close()    
df = pd.DataFrame(Name_list)
save_photos(df) 


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\Noam\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\Noam\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [25]:
df.drop_duplicates(subset=['Name'],inplace=True)
df.to_csv("project_data_gemstones_table.csv")

In [31]:
df

,Name,Formula,Elements,Colours,Image_url,Locations,Locations_url
0,Amazonite,K(AlSi3O8),"Al, K, O, Si",NaN,['/imagecache/11/7a/04766080014946246016426.jp...,"['Argentina', '\xa0', 'Australia', '\xa0', 'Au...","['https://www.mindat.org//loc-269722.html', 'h..."
1,Amethyst,SiO2,"O, Si",Violet - purpleC,['/imagecache/8a/ca/03039750014946246163278.jp...,"['Afghanistan', '\xa0', 'Angola', '\xa0', 'Arg...","['https://www.mindat.org//loc-404929.html', 'h..."
2,Ametrine,SiO2,"O, Si",NaN,['/imagecache/d2/c6/04532760014946392196542.jp...,"['Bolivia', '\xa0', 'Brazil', '\xa0', 'Canada'...","['https://www.mindat.org//loc-11804.html', 'ht..."
3,Ammolite,NaN,NaN,NaN,['/imagecache/b4/a9/00207230014946603988222.jp...,"['Austria', '\xa0', 'Canada', '\xa0', 'Madagas...","['https://www.mindat.org//loc-205904.html', 'h..."
4,Andalusite,Al2(SiO4)OIMA Formula:Al2SiO5,Bonfim W,"Pink to red brown, occasionally yellow, green,...",['/imagecache/ff/30/01398870014946330614953.jp...,"['Andorra', '\xa0', 'Antarctica', 'AmMin 36:36...","['https://www.mindat.org//loc-31239.html', 'ht..."
...,...,...,...,...,...,...,...
73,Turquoise,CuAl6(PO4)4(OH)8 · 4H2O,"Al, Cu, H, O, P","bright blue, sky-blue, pale green, blue-green,...",['/imagecache/6d/d1/05330790014956573955192.jp...,"['Afghanistan', 'Gübelin, E., Wolgensinger, M....","['https://www.mindat.org/loc-14319.html', 'htt..."
74,Unakite,NaN,NaN,NaN,['/imagecache/5a/7d/05648390015601981974323.jp...,[],[]
75,Variscite,AlPO4 · 2H2OIMA Formula:Al(PO4) · 2H2O,"Sn–W–Pb–Ag–Zn Huanuni Deposit, Bolivia. Minera...","Pale to emerald-green, bluish green, colourles...",['/imagecache/77/23/08123090014977192376273.jp...,"['Antigua and Barbuda', '\xa0', 'Argentina', '...","['https://www.mindat.org//loc-147083.html', 'h..."
76,Zircon,"Zr(SiO4)May contain minor U, Th, Pb, Hf, Y/REE...",", Pr, Nd, and Sm) from a Carbonatite Deposit: ...","Colourless, yellow, grey, reddish-brown, green...",['/imagecache/40/37/05576060014946320573225.jp...,"['Afghanistan', '\xa0', 'Algeria', '\xa0', 'An...","['https://www.mindat.org//loc-3.html', 'https:..."


In [37]:
def clean_loc(loc):
    if(loc.find("'")!=-1):
        loc = loc.replace("'",'',1)
    if(loc.find("]")!=-1):
        loc = loc.replace("]",'',1)
    if(loc.find("[")!=-1):
        loc = loc.replace("[",'',1)
    if(loc.find(",")!=-1):
        loc = loc.replace(",",'',1)
    if(loc.find("'")!=-1):
        loc = loc.replace("'",'',1)
    return loc

In [38]:
df = pd.read_csv("project_data_gemstones_table.csv")
locs = df['Locations_url']
locs = pd.DataFrame(locs)
new_locs = []
for i in locs.index:
    for string in locs['Locations_url'][i].split():
        string = clean_loc(string)
           
        new_locs.append(string)
new_locs = pd.DataFrame(new_locs)


In [39]:
for i in new_locs.index:
    if(i%10!=0):
        new_locs = new_locs.drop(index = i)

In [40]:
new_locs = new_locs.drop_duplicates().reset_index()
new_locs = pd.DataFrame(new_locs[0])
new_locs.drop_duplicates()
new_loc_df = pd.DataFrame(index=new_locs.index, columns=["Location_URL","Stone_Name"])

dict_locs = {}

for ind in new_locs.index:
    if(ind%2==0):continue
    for i in locs.index:

        stones_for_loc = []
        curr_loc = new_locs[0][ind]

        for string in locs['Locations_url'][i].split():
            if(string.find(curr_loc)!=-1):
                stones_for_loc.append(df['Name'][i])
                try:
                    dict_locs[string] +=" " + df['Name'][i]
                except KeyError:
                    dict_locs[string] = df['Name'][i]


In [41]:
index = []
for i in dict_locs:
    index.append(i)
index
new_loc_df = pd.DataFrame(dict_locs.values(), columns=["Stones_Name"])
new_loc_df["URL"] = index
new_loc_df

,Stones_Name,URL
0,Amazonite,"'https://www.mindat.org//loc-63988.html',"
1,Amazonite Scapolite Sphene Titanite,"'https://www.mindat.org//loc-21257.html',"
2,Amazonite,"'https://www.mindat.org//loc-272378.html',"
3,Amazonite Beryl Diopside Serpentine Sphene Spi...,"'https://www.mindat.org//loc-9525.html',"
4,Amazonite Amethyst Beryl Diopside,"'https://www.mindat.org//loc-133112.html',"
...,...,...
3616,Zoisite,"'https://www.mindat.org//loc-17176.html',"
3617,Amethyst Apatite Kyanite Sphene Titanite Zirco...,"'https://www.mindat.org//loc-30133.html',"
3618,Diopside Sphene Titanite Zoisite,"'https://www.mindat.org//loc-122349.html',"
3619,Apatite Diopside Zoisite,"'https://www.mindat.org//loc-38903.html',"


In [55]:
new_loc_df.to_csv("location_and_stones_table.csv")

In [111]:
def find_mineral_commodities_recorded(loc_driver):
    minerals = ""
    try:
        soup = BeautifulSoup(loc_driver.page_source, 'html.parser')
        minerals_soup = soup.find("div",{"id":"ltmain"})
        minerals = soup.find("div",{"id":"newloccomlist"})
        minerals = minerals.text
    except NoSuchElementException:
        pass
    except AttributeError:
        pass
    return minerals


def find_loc_name(loc_soup):
    time.sleep(1)
    Name = ""
    try:
        h1 = loc_soup.find("div",{"class":"fpbox990nl"})
        names = h1.find_all("a")
        i = 1
        for name in names:
            Name +=name.text + ","
            i += 1
    except NoSuchElementException:
        pass
    except AttributeError:
        pass
    return Name


def get_minerals(loc_driver, loc_soup):
    time.sleep(1)
    minerals = []
    try:
        div = loc_soup.find("div",{"id":"filters"})
        span = div.find_all("span")
        for sp in span:
            minerals.append(sp.text)
    except NoSuchElementException:
        pass
    except AttributeError:
        pass

        
    return minerals

def find_rock_types(loc_soup):
    rock_type_list = []
    try:
        rock_types = loc_soup.find("div",{"id":"newlocrocklist"})
        rt = rock_types.find_all("a")
        for r in rt:
            a=r.text
            rock_type_list.append(a)
    except NoSuchElementException:
        pass
    except AttributeError:
        pass
    return rock_type_list

def find_locations_details(new_loc_df, loc_driver):
    total = []
    for i in new_loc_df.index:
        time.sleep(1)
        loc = clean_loc(new_loc_df['URL'][i])
        if(loc.find("https://www.mindat.org/loc-293106.html")!=-1): continue
        try:
            loc_soup = BeautifulSoup(loc_driver.page_source,'html.parser')
            Dec_cords_list = ""
            loc_climate = ""
            Coordinates  = loc_soup.find_all("div",{"class":"LFtd"})
            for cor in Coordinates:
                cor = cor.text
                if(cor.find(" : ")!=-1):###find climate
                    loc_climate= cor
                else:
                    if(any(c.isalpha() for c in cor)==False): ##check for cords digits only
                        Dec_cords_list=cor
                    
        except NoSuchElementException: 
            warn = webdriver.Chrome(ChromeDriverManager().install())
            time.sleep(10)
            loc_driver.close()
            warn.close()
            loc_driver = webdriver.Chrome(ChromeDriverManager().install())
            loc_driver.get(loc)
            pass
        except WebDriverException:
            loc_driver.close()
            loc_driver = webdriver.Chrome(ChromeDriverManager().install())
            time.sleep(10)
            loc_driver.get(loc)
            pass
        except TypeError:
            loc_driver.close()
            loc_driver = webdriver.Chrome(ChromeDriverManager().install())
            time.sleep(10)
            loc_driver.get(loc)
            pass
            
        loc_data_dict = {}
        loc_data_dict["Location URL"] = loc
        loc_data_dict["Stone Refernced"] = new_loc_df['Stones_Name'][i]
        loc_data_dict["Minerals Recorded"] = find_mineral_commodities_recorded(loc_driver)
        loc_driver.get(loc)
        time.sleep(1)
        loc_name=find_loc_name(loc_soup)
        loc_data_dict["Location Name"] = loc_name
        loc_data_dict["Rock Types"] = find_rock_types(loc_soup)
        
        loc_data_dict["Climate"] = loc_climate
        loc_data_dict["Decimal Coordinates"] = Dec_cords_list
        loc_data_dict["Minerals"] = get_minerals(loc_driver, loc_soup)
        if(len(Dec_cords_list)==0):continue
        total.append(loc_data_dict)
    return total



In [90]:
loc_driver = webdriver.Chrome(ChromeDriverManager().install())
t = find_locations_details(new_loc_df, loc_driver)
loc_driver.close()
datf = pd.DataFrame(t)



[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\Noam\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [92]:
datf

,Location URL,Stone Refernced,Minerals Recorded,Location Name,Rock Types,Climate,Decimal Coordinates,Minerals
0,https://www.mindat.org//loc-4047.html,Amazonite Apatite Aquamarine Beryl Opal Smoky ...,None,", Preitenegg, Wolfsberg District, Carinthia, A...",[],Dfb : Warm-summer humid continental climate,"46.96177,14.95181","[Valid Species, Al, C, Ca, Ce, F, Fe, H, K, La..."
1,https://www.mindat.org//loc-250161.html,Amazonite Aquamarine Beryl Kyanite Smoky Quartz,None,", Spruce Pine, Spruce Pine Mining District, Mi...",[],Cfb : Temperate oceanic climate,"35.91667,-82.06667","[Al, B, Be, C, Ca, Cd, Ce, Cl, Cu, F, Fe, H, K..."
2,https://www.mindat.org//loc-8991.html,Amethyst Apatite Spinel,None,", Thanh Son District, Phú Thọ Province, Vietna...",[],Cwa : Monsoon-influenced humid subtropical cli...,"21.18306,105.25000","[Al, B, Be, F, Fe, H, K, Mg, O, Si, Ti]"
3,https://www.mindat.org//loc-56146.html,Amethyst,None,", Eggenburg, Waldviertel, Lower Austria, Austr...",[],Cfb : Temperate oceanic climate,"48.64833,15.82083","[Valid Species, O, Si]"
4,https://www.mindat.org//loc-232658.html,Amethyst,None,", Krimml, Zell am See District, Salzburg, Aust...",[],Dfb : Warm-summer humid continental climate,"47.15161,12.13835","[Valid Species, O, Si]"
...,...,...,...,...,...,...,...,...
1599,https://www.mindat.org//loc-193574.html,Diopside Zoisite,None,", Cochise Mining District, Little Dragoon Moun...",[],BSk : Cold semi-arid (steppe) climate,"32.06556,-110.04056","[Valid Species, Al, Ca, Cu, Fe, H, K, Mo, O, S..."
1600,https://www.mindat.org//loc-17150.html,Diopside Zoisite,ⓘ Sand and Gravel,", Palm Springs, Riverside Co., California, USA...",[],Csa : Hot-summer Mediterranean climate,"33.89747,-116.53778","[Al, B, C, Ca, F, Fe, H, K, Mg, Mn, Na, O, Si,..."
1601,https://www.mindat.org//loc-6134.html,Apatite Cordierite Kyanite Zoisite,None,", Lyon Co., Nevada, USA,i,Silver City Mining D...",[],Csb : Warm-summer Mediterranean climate,"39.26361,-119.64056","[Ag, Al, As, Au, B, C, Ca, Cu, Fe, H, K, Mg, M..."
1602,https://www.mindat.org//loc-17176.html,Zoisite,ⓘ Feldspar,", Mitchell Co., North Carolina, USA,i,Pine Mou...",[],Cfb : Temperate oceanic climate,"35.94305,-82.09000","[Al, Ca, Cu, F, Fe, H, K, Mg, Mn, O, P, Si, Ti..."


In [97]:
datf.to_csv("project_data_location_table1.csv")

In [112]:
israel_locations_list = ["https://www.mindat.org/loc-14762.html"]
israel_driver = webdriver.Chrome(ChromeDriverManager().install())
israel_driver.get("https://www.mindat.org/loc-14762.html")
israel_soup = BeautifulSoup(israel_driver.page_source,'html.parser')
israel_locs = israel_soup.find("div",{"class":"loclistcontainer"})
mt = israel_locs.find_all("a")
for m in mt:
    a="https://www.mindat.org/"+m.get("href")
    if(a.find("/loc-")!=-1):
        israel_locations_list.append(a)
    


israel_loc_df = pd.DataFrame(columns=["Stones_Name"])
israel_loc_df["URL"] = israel_locations_list
israel_loc_df

israel_t = find_locations_details(israel_loc_df, israel_driver)
israel_driver.close()
israel_df = pd.DataFrame(israel_t)




[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Noam\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [113]:
israel_df

,Location URL,Stone Refernced,Minerals Recorded,Location Name,Rock Types,Climate,Decimal Coordinates,Minerals
0,https://www.mindat.org/loc-14762.html,NaN,ⓘ Copperⓘ Gemstones,",i,Israel,Country,Click here to sponsor this p...","[ⓘ, Alkali basalt, ⓘ, Analcimite, ⓘ, Basalt, ⓘ...",,,"[Ag, Al, B, Ba, C, Ca, Ce, Cl, Cr, Cu, F, Fe, ..."
1,https://www.mindat.org/loc-14762.html,NaN,ⓘ Copperⓘ Gemstones,",i,Israel,Country,Click here to sponsor this p...","[ⓘ, Alkali basalt, ⓘ, Analcimite, ⓘ, Basalt, ⓘ...",,,"[Ag, Al, B, Ba, C, Ca, Ce, Cl, Cr, Cu, F, Fe, ..."
2,https://www.mindat.org//loc-145487.html,NaN,ⓘ Copperⓘ Gemstones,",i,Israel,Country,Click here to sponsor this p...","[ⓘ, Alkali basalt, ⓘ, Analcimite, ⓘ, Basalt, ⓘ...",,,"[Ag, Al, B, Ba, C, Ca, Ce, Cl, Cr, Cu, F, Fe, ..."
3,https://www.mindat.org//loc-145486.html,NaN,None,", Israel,i,Central District (HaMerkaz District...",[],,,[]
4,https://www.mindat.org//loc-145489.html,NaN,ⓘ Gemstones,", Israel,i,Haifa District,District, Israel,Cou...","[ⓘ, Alkali basalt, ⓘ, Kimberlite, ⓘ, Peridotit...",,,"[Al, B, C, Ca, Cl, Cr, F, Fe, H, K, Mg, N, Na,..."
5,https://www.mindat.org//loc-145488.html,NaN,None,", Israel,i,Jerusalem District,District, Israel...",[],,,"[Al, C, Ca, H, O, Si]"
6,https://www.mindat.org//loc-205351.html,NaN,None,", Israel,i,Northern District (HaZafon District...","[ⓘ, Basalt, ⓘ, 'Dolerite', ⓘ, 'Melanocratic ne...",,,"[Al, C, Ca, Cl, Fe, H, K, Mg, Mn, Na, Ni, O, S..."
7,https://www.mindat.org/loc-14762.html,NaN,ⓘ Copper,", Israel,i,Southern District (HaDarom District...","[ⓘ, Analcimite, ⓘ, Basalt, ⓘ, Basanite, ⓘ, Bre...",,,"[Ag, Al, B, Ba, C, Ca, Ce, Cl, Cr, Cu, F, Fe, ..."
8,https://www.mindat.org//loc-205351.html,NaN,ⓘ Copperⓘ Gemstones,",i,Israel,Country,Click here to sponsor this p...","[ⓘ, Alkali basalt, ⓘ, Analcimite, ⓘ, Basalt, ⓘ...",,,"[Ag, Al, B, Ba, C, Ca, Ce, Cl, Cr, Cu, F, Fe, ..."
9,https://www.mindat.org//loc-16782.html,NaN,ⓘ Copper,", Israel,i,Southern District (HaDarom District...","[ⓘ, Analcimite, ⓘ, Basalt, ⓘ, Basanite, ⓘ, Bre...",,,"[Ag, Al, B, Ba, C, Ca, Ce, Cl, Cr, Cu, F, Fe, ..."


In [120]:
loc_df = pd.concat([israel_df,datf])
loc_df = loc_df.drop_duplicates(subset=['Location URL','Location Name'])
loc_df = loc_df.reset_index(drop = True)
loc_df

,Location URL,Stone Refernced,Minerals Recorded,Location Name,Rock Types,Climate,Decimal Coordinates,Minerals
0,https://www.mindat.org/loc-14762.html,NaN,ⓘ Copperⓘ Gemstones,",i,Israel,Country,Click here to sponsor this p...","[ⓘ, Alkali basalt, ⓘ, Analcimite, ⓘ, Basalt, ⓘ...",,,"[Ag, Al, B, Ba, C, Ca, Ce, Cl, Cr, Cu, F, Fe, ..."
1,https://www.mindat.org//loc-145487.html,NaN,ⓘ Copperⓘ Gemstones,",i,Israel,Country,Click here to sponsor this p...","[ⓘ, Alkali basalt, ⓘ, Analcimite, ⓘ, Basalt, ⓘ...",,,"[Ag, Al, B, Ba, C, Ca, Ce, Cl, Cr, Cu, F, Fe, ..."
2,https://www.mindat.org//loc-145486.html,NaN,None,", Israel,i,Central District (HaMerkaz District...",[],,,[]
3,https://www.mindat.org//loc-145489.html,NaN,ⓘ Gemstones,", Israel,i,Haifa District,District, Israel,Cou...","[ⓘ, Alkali basalt, ⓘ, Kimberlite, ⓘ, Peridotit...",,,"[Al, B, C, Ca, Cl, Cr, F, Fe, H, K, Mg, N, Na,..."
4,https://www.mindat.org//loc-145488.html,NaN,None,", Israel,i,Jerusalem District,District, Israel...",[],,,"[Al, C, Ca, H, O, Si]"
...,...,...,...,...,...,...,...,...
1612,https://www.mindat.org//loc-193574.html,Diopside Zoisite,None,", Cochise Mining District, Little Dragoon Moun...",[],BSk : Cold semi-arid (steppe) climate,"32.06556,-110.04056","[Valid Species, Al, Ca, Cu, Fe, H, K, Mo, O, S..."
1613,https://www.mindat.org//loc-17150.html,Diopside Zoisite,ⓘ Sand and Gravel,", Palm Springs, Riverside Co., California, USA...",[],Csa : Hot-summer Mediterranean climate,"33.89747,-116.53778","[Al, B, C, Ca, F, Fe, H, K, Mg, Mn, Na, O, Si,..."
1614,https://www.mindat.org//loc-6134.html,Apatite Cordierite Kyanite Zoisite,None,", Lyon Co., Nevada, USA,i,Silver City Mining D...",[],Csb : Warm-summer Mediterranean climate,"39.26361,-119.64056","[Ag, Al, As, Au, B, C, Ca, Cu, Fe, H, K, Mg, M..."
1615,https://www.mindat.org//loc-17176.html,Zoisite,ⓘ Feldspar,", Mitchell Co., North Carolina, USA,i,Pine Mou...",[],Cfb : Temperate oceanic climate,"35.94305,-82.09000","[Al, Ca, Cu, F, Fe, H, K, Mg, Mn, O, P, Si, Ti..."


In [125]:
loc_df.to_csv("raw_project_data_location_table.csv")